In [2]:
import numpy as np
np.add(2,40)

42

In [ ]:
#Code for Text Generation RNN #https://gist.github.com/karpathy/d4dee566867f8291f086

import numpy as np

#Data Processing

data = open('input.txt', 'r').read()
chars = list(set(data)) #Get all unique chars
data_size, vocab_size = len(data), len(chars)
print('Data has %d chars with %d unique' %(data_size, vocab_size))
#Assign chars to indices and vice-versa. This will be needed to go between chars and indices
char_to_ix = {ch:i for i,ch in enumerate(chars)} # Creates a dict, enumurate loops through list "chars" 
ix_to_char = {i:ch for i,ch in enumerate(chars)} # and returns an index as well as the elements of the list

#Hyperparameters
HIDDEN_SIZE = 100 # Units in the RNN unit's hidden layer
SEQ_LENGTH = 25  #RNN will unroll this much
LEARNING_RATE = 1e-1

#Model Weights and Biases
# We have 1 RNN unit with a Hidden Size of 100, which is going to be unrolled 25 times
# Input will be a 1-hot encoded char , so will the output
# Eg. if 26 uniq chars, then 26 -> 100 -> 26 in I/O axis
# and 100 -> 100 in the temporal axis
#
#  26    26
#  ^      ^
# 100 > 100 > (25 times)
#  ^     ^
#  26   26
# This 26 is used as an example but should be replaced by number of unique chars instead



# These 5 things are what actually get trained
W_xh = np.random.randn(HIDDEN_SIZE, vocab_size)*0.01  # H <- I, randn ~ N(0,1)
W_hh = np.random.randn(HIDDEN_SIZE, HIDDEN_SIZE)*0.01 # H <- H
W_hy = np.random.randn(vocab_size, HIDDEN_SIZE)*0.01  # O <- H

b_h = np.zeros((HIDDEN_SIZE, 1)) # Bias for hidden layer
b_y = np.zeros((vocab_size, 1))  # Bias for output layer

# We have only 2 bias terms, though 3 weight matrices. That's because we calculate the 


#Now we define the loss function

def lossFun(inputs, targets, hprev):
    
    # inputs and targets are indices i.e. integers of input and target chars, not 1 hot encoding yet
    # hprev are the outputs of the hidden layer, list of HIDDEN_SIZE x 1, since 1 ouput per hidden unit
    
    xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
    
    hs[-1] = np.copy(hprev) # Using "=" would create a reference, this creates a whole separate copy
                            # We don't want hs[-1] to automatically change if hprev is changed
    
    loss = 0
    
    # We now do the Forward Pass
    # We run a loop for SEQ_LENGTH time steps and populate the dicts
    
    for t in range(len(inputs)):
        
        xs[t] = np.zeros((vocab_size, 1)) # Here we place a 0 vector as the t-th input
        xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
        # index to 1. These 2 lines basically create the 1 hot-encoded vector of the character at t-th timestep
        
        hs[t] = np.tanh( np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h)
        # hs has outputs of hidden layer. To get this we have to add three things:
        # First the pre-activation coming from the inputs  np.dot(W_xh, xs[t])
        # Second the pre-activation coming from the prev hidden state  np.dot(W_hh, hs[t-1])
        # At t=0, this will become hs[-1]. That's why we gave it hprev to get the ball rolling
        # Third, the bias term
        # These 3 are summed and passed through the activation function tanh
        # Inside the tanh, we are summing (100x26 dot 26x1) + (100x100 dot 100x1) + (100x1)
        # Each of the 3 terms ends up in 100x1 vectors that are suitable to be summed with each other
        
        ys[t] = np.dot(W_hy, hs[t]) + b_y # To get raw outputs (26x100 dot 100x1) + (26x1)
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # Convert to probabs over the entire char setusing softmax
        # ys[t] is 26x1 vector with 26 numbers whose exponential we take to get another 26 numbers
        # We want each of those 26 numbers to be divided by the sum of all those 26 numbers
        
        loss += -np.log(ps[t][targets[t], 0]) 
        # We will now calculate the loss. This loss is a running loss over the entire input sequence of 25 chars
        # ps is a dict, with keys being time steps and the values being the probab distb at that time 
        # ps[t], though having just 26 numbers is actually a 2D numpy matrix, because ys was a 26x1 matrix 
        # (which is 2D in the sense that it has 0 axis and 1 axis)
        # ps[t][targets[t], 0] first goes to the target character's index and then chooses the first (and only)
        # value in it. ps[t] looks something like [ [0.1], [0.4], .... 26 entries]
        # To access 0.4, we must to ps[t][2, 0]
        # Doing just ps[t][2] will give is [0.4] which is an (1,) array and not scalar 0.4
        # loss has to be a scalar loss
        # targets[t] is the correct character's index value (and not one-hot encoded version)
        
        
        
        
    
    


Variable        Type        Data/Info
-------------------------------------
Whh             ndarray     100x100: 10000 elems, type `float64`, 80000 bytes
Why             ndarray     77x100: 7700 elems, type `float64`, 61600 bytes
Wxh             ndarray     100x77: 7700 elems, type `float64`, 61600 bytes
bh              ndarray     100x1: 100 elems, type `float64`, 800 bytes
by              ndarray     77x1: 77 elems, type `float64`, 616 bytes
ch              str          
char_to_ix      dict        n=77
chars           list        n=77
dWhh            ndarray     100x100: 10000 elems, type `float64`, 80000 bytes
dWhy            ndarray     77x100: 7700 elems, type `float64`, 61600 bytes
dWxh            ndarray     100x77: 7700 elems, type `float64`, 61600 bytes
data            str         \nCHAPTER I\n\nDhritirash<...> how with God he spake.\n
data_size       int         110549
dbh             ndarray     100x1: 100 elems, type `float64`, 800 bytes
dby             ndarray     77x1: 77 elems, type `float64`, 616 bytes
dparam          ndarray     77x1: 77 elems, type `float64`, 616 bytes
hidden_size     int         100
hprev           ndarray     100x1: 100 elems, type `float64`, 800 bytes
inputs          list        n=25
ix_to_char      dict        n=77
learning_rate   float       0.1
loss            float64     75.6018751803
lossFun         function    <function lossFun at 0x7fcaccfada28>
mWhh            ndarray     100x100: 10000 elems, type `float64`, 80000 bytes
mWhy            ndarray     77x100: 7700 elems, type `float64`, 61600 bytes
mWxh            ndarray     100x77: 7700 elems, type `float64`, 61600 bytes
mbh             ndarray     100x1: 100 elems, type `float64`, 800 bytes
mby             ndarray     77x1: 77 elems, type `float64`, 616 bytes
mem             ndarray     77x1: 77 elems, type `float64`, 616 bytes
n               int         367
np              module      <module 'numpy' from '/ho<...>ages/numpy/__init__.pyc'>
p               int         9175
param           ndarray     77x1: 77 elems, type `float64`, 616 bytes
sample          function    <function sample at 0x7fca9f0e6578>
sample_ix       list        n=200
seq_length      int         25
smooth_loss     float64     101.765250884
targets         list        n=25
txt             str         n f c nf foe nin ean nBil<...>rsa\nRtra h rfe b, dryt r
vocab_size      int         77


In [17]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in xrange(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    print(ps[t][targets[t]].shape)
    print("*******")
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in xrange(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print '----\n %s \n----' % (txt, )

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 


data has 110549 characters, 77 unique.
----
 78dEcfSSsN'upXlz(IgqfaGNWRN7'bAHdCzjTpBH)jXjWbkAx-gd2-]XtSA8"1".x3f2AE
?meECQCT.aIvzE1kaDEvfjYecaojUR74yX3ixThg7[9yNs?fuvV?#O"J
H[.TUAFY
 '?kU?47-4?,VBx,M994J:ho23F9OedT3]m"CglYH1HaC#fxmH0TrIwUjjpccUV 
----
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
iter 0, loss: 108.595125
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
*******
(1,)
***

KeyboardInterrupt: 

In [2]:
%whos

Variable        Type        Data/Info
-------------------------------------
Whh             ndarray     100x100: 10000 elems, type `float64`, 80000 bytes
Why             ndarray     77x100: 7700 elems, type `float64`, 61600 bytes
Wxh             ndarray     100x77: 7700 elems, type `float64`, 61600 bytes
bh              ndarray     100x1: 100 elems, type `float64`, 800 bytes
by              ndarray     77x1: 77 elems, type `float64`, 616 bytes
ch              str          
char_to_ix      dict        n=77
chars           list        n=77
dWhh            ndarray     100x100: 10000 elems, type `float64`, 80000 bytes
dWhy            ndarray     77x100: 7700 elems, type `float64`, 61600 bytes
dWxh            ndarray     100x77: 7700 elems, type `float64`, 61600 bytes
data            str         \nCHAPTER I\n\nDhritirash<...> how with God he spake.\n
data_size       int         110549
dbh             ndarray     100x1: 100 elems, type `float64`, 800 bytes
dby             ndarray     77x1: 7

In [7]:
print(vars(Whh))

TypeError: vars() argument must have __dict__ attribute

In [8]:
from pprint import pprint
#pprint(globals())
pprint(locals())

{'In': ['',
        u'"""\nMinimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)\nBSD License\n"""\nimport numpy as np\n\n# data I/O\ndata = open(\'input.txt\', \'r\').read() # should be simple plain text file\nchars = list(set(data))\ndata_size, vocab_size = len(data), len(chars)\nprint \'data has %d characters, %d unique.\' % (data_size, vocab_size)\nchar_to_ix = { ch:i for i,ch in enumerate(chars) }\nix_to_char = { i:ch for i,ch in enumerate(chars) }\n\n# hyperparameters\nhidden_size = 100 # size of hidden layer of neurons\nseq_length = 25 # number of steps to unroll the RNN for\nlearning_rate = 1e-1\n\n# model parameters\nWxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden\nWhh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden\nWhy = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output\nbh = np.zeros((hidden_size, 1)) # hidden bias\nby = np.zeros((vocab_size, 1)) # output bias\n\ndef lossFun(inputs, targets

In [14]:
np.zeros( (1) ).shape

(1,)